*Search Engine with Tools & Agents*

In [1]:
### arxiv = For research articles and interaction with arxiv
## Tools - creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
## using inbuilt tool pf wikipedia

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,
                                       doc_content_chars_max= 250) # fetches page summaries

wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,
                                    doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [4]:
#tools made n defined - inbuilt
tools = [wiki,arxiv]

In [7]:
## custom tools by us - rag tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
!pip install python-dotenv

You should consider upgrading via the 'C:\Users\hp\VSCode\PYTHON\Gen-AI-Practice\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [22]:
# Load env vars
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
import os
print("🔐", os.getenv("OPENAI_API_KEY")[:10])

🔐 sk-irm4HZG


In [9]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,
                                         chunk_overlap=200).split_documents(docs)

In [15]:
vectordb = FAISS.from_documents(documents,OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002A115F3B0A0>, search_kwargs={})

In [ ]:
# creation of our own tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,
                                      "my-langsmith-search",
                                       "Search about transformer models")
retriever_tool.name

'my-langsmith-search'

In [18]:
tools = [wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\hp\\VSCode\\PYTHON\\Gen-AI-Practice\\.venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='my-langsmith-search', description='Search about transformer models', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002A150146290>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002A

# run all tools with agents and LLM model

In [29]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()
import openai

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key , model="Llama3-8b-8192")

In [30]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002A1537769B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002A1537770A0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [31]:
## prompt template

from langchain import hub # prompt created and uploaded to hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [33]:
# combine prompt with llm with tools created - 
# will be executed altogether in form of chains using agent executer

from langchain.agents import create_openai_tools_agent

agent =  create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [34]:
# agent executer for running the agent created

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [35]:
agent_executor.invoke({"input":"tell me about attention mechanism in nutshell"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'attention mechanism'}`


Page: Attention (machine learning)
Summary: In machine learning, attention is a  method that determines the importance of each component in a sequence relative to the other components in that sequence. In natural language processing, importance is reAccording to the result of the tool call, it seems that attention mechanism is a method in machine learning that determines the importance of each component in a sequence relative to the other components in that sequence. In natural language processing, this is particularly important for tasks such as text summarization, machine translation, and question answering.

Here's a more detailed explanation:

Attention mechanism is a technique used in deep learning models to help them focus on the most relevant parts of an input sequence. This is particularly useful when dealing with long sequences, such as sentences or paragraphs, where not al

{'input': 'tell me about attention mechanism in nutshell',
 'output': "According to the result of the tool call, it seems that attention mechanism is a method in machine learning that determines the importance of each component in a sequence relative to the other components in that sequence. In natural language processing, this is particularly important for tasks such as text summarization, machine translation, and question answering.\n\nHere's a more detailed explanation:\n\nAttention mechanism is a technique used in deep learning models to help them focus on the most relevant parts of an input sequence. This is particularly useful when dealing with long sequences, such as sentences or paragraphs, where not all parts are equally important.\n\nIn traditional recurrent neural networks (RNNs), all parts of the sequence are processed equally, which can lead to the model losing focus on the most important parts. The attention mechanism addresses this issue by allowing the model to weigh th

In [37]:
agent_executor.invoke({"input":"explain langraph and langchain in simple words"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'What is Langraph and Langchain in simple words?'}`


No good Wikipedia Search Result was found
Invoking: `wikipedia` with `{'query': 'Langraph and Langchain explained in simple words'}`


No good Wikipedia Search Result was found
Invoking: `my-langsmith-search` with `{'query': 'What is Langraph and Langchain?'}`


Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your appli

{'input': 'explain langraph and langchain in simple words',
 'output': 'It seems like the tool call yielded a webpage content related to LangGraph and LangChain. Let me try to summarize the information in simple words:\n\nLangGraph and LangChain are tools used to build and develop Large Language Models (LLMs) applications. LangGraph is a platform for building ambient agents, which are AI systems that can understand and respond to natural language inputs. LangChain is an open-source framework that allows developers to build LLM applications.\n\nThe content also mentions LangSmith, which is a platform for building production-grade LLM applications. It has features such as observability, evals, and prompt engineering, which help developers debug, test, and optimize their applications.\n\nIn simple terms, LangGraph, LangChain, and LangSmith are tools that help developers build and improve AI systems that can understand and respond to human language.'}

In [ ]:
#checking if it consider its previous responses i.e. chat history
agent_executor.invoke({"input":"is transformer knowledge important to run llm models in langraph?"})



> Entering new AgentExecutor chain...

Invoking: `my-langsmith-search` with `{'query': 'Is transformer knowledge important to run LLM models in Langraph?'}`


Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performan

{'input': 'is transformer knowledge important to run llm models in langraph?',
 'output': "Based on the result from the tool call, it seems that transformer knowledge is not explicitly mentioned as a requirement for running LLM models in Langraph. However, it does mention that LangSmith provides LLM-native observability, which might be important for debugging and optimizing LLM applications.\n\nHere's a response that directly answers the user's question:\n\nTransformer knowledge is not explicitly mentioned as a requirement for running LLM models in Langraph. However, LangSmith provides LLM-native observability, which can help with debugging and optimizing LLM applications."}

In [ ]:
# from openai import OpenAI
# client = OpenAI(api_key="...")

# response = client.responses.create(
#     model="gpt-4.1-nano-2025-04-14",
#     input="example of langchain architecture component tool?"
# )

# print(response.output_text)

Certainly! In LangChain, **Tools** are components that provide external functionalities or integrations to augment your language chain's capabilities. They are used within chains or agents to perform specific tasks such as searching, executing code, or accessing external APIs.

### Example of a LangChain Tool

One common example is an **"Elicit Search Tool"** that allows an agent to perform web searches or retrieve information from external sources.

Here's a simplified example of how a **Search Tool** might be implemented in LangChain:

```python
from langchain.tools import BaseTool
import requests

class SearchTool(BaseTool):
    name = "search"
    description = "Use this tool to perform web searches and retrieve relevant information."

    def _run(self, query: str):
        # For example, using an API like Bing Search API
        api_key = "YOUR_API_KEY"
        headers = {"Ocp-Apim-Subscription-Key": api_key}
        params = {"q": query, "count": 3}
        response = requests.g